# Import Data

## Présentation
Lorsqu'il est nécessaire de mettre à jour la base de données MongoDB avec de nouvelles données, il peut être pratique de supprimer tous les documents (Tweets) de la collection existante et d'insérer les nouveaux documents à partir des fichiers JSON contenu dans le dossier ``data`` à la racine du projet pour garantir la cohérence des données et s'assurer que nous ne disposons pas de données modifiées. Dans ce cas-ci, ce script peut être utilisé.

Que fait-il ?
1. Suppression des données existantes dans la base de données MongoDB ayant pour nom Tweet
2. Lecture des données présente dans le dossier ``data`` à la racine du projet. A noter que nous n'avons pas commit ces données à cause d'une taille volumineuse. Elles peuvent être téléchargées via le lien suivant : [Cliquer ici](https://www.dropbox.com/s/qfhaobip55xxkif/Tweet%20Worldcup.zip?dl=0)
3. Insertions des données lues dans la base de données

## Le code
On importe les différentes bibliothèques nécessaires.

In [1]:
import pymongo
import json
import os
from datetime import datetime
import numpy as np

Connexion à la base de données MongoDB

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
dataset_collection = db["dataset"]

Suppression de tous les documents de la collection

In [3]:
dataset_collection.drop()

Définition de la fonction pour upload dans mongodb tout les documents contenus dans un fichier

In [4]:
def upload_file_data(path):
    with open(path, encoding='UTF-8') as f:
        docs = []
        for line in f:
            doc = json.loads(line)
            doc["created_date"] = datetime.strptime(doc["created_date"],"%Y-%m-%d %H:%M:%S")
            doc["user"]["created_at"] = datetime.strptime(doc["user"]["created_at"], "%a %b %d %H:%M:%S %z %Y")
            docs.append(doc)
        if len(docs) > 0:
            dataset_collection.insert_many(docs)


Récupération des noms des fichiers JSON dans le dossier approprié

In [5]:
path = "../../data"
all_files = os.listdir(path)

On parcourt l'ensemble des fichiers pour upload les documents qui le constitue tout en affichant des statistiques pour suivre l'évolution de l'importation des données dans MongoDB

In [6]:
start_time = datetime.now()
print(f">>>> Start : {start_time}")

# begins upload
nb_files = len(all_files)
for idx_files, file in enumerate(all_files):
    start_file_time = datetime.now() 
    if file.endswith(".json"):
        upload_file_data(f"{path}/{file}")
    time_for_this_file = (datetime.now()  - start_file_time).seconds
    percentage = round(idx_files*100/nb_files,2)
    print(f" - Upload files n°{idx_files}/{nb_files} in {time_for_this_file}s ({percentage}%)")

# some stats
end_time = datetime.now()     
print(f">>>> End : {end_time}")
diff_time = end_time - start_time
print(f"Upload done in {diff_time.seconds // 3600}h {(diff_time.seconds // 60) % 60}min {diff_time.seconds % 60}sec ")

>>>> Start : 2023-06-13 10:30:40.763241
 - Upload files n°0/2287 in 0s (0.0%)
 - Upload files n°1/2287 in 0s (0.04%)
 - Upload files n°2/2287 in 0s (0.09%)
 - Upload files n°3/2287 in 0s (0.13%)
 - Upload files n°4/2287 in 0s (0.17%)
 - Upload files n°5/2287 in 0s (0.22%)
 - Upload files n°6/2287 in 0s (0.26%)
 - Upload files n°7/2287 in 0s (0.31%)
 - Upload files n°8/2287 in 0s (0.35%)
 - Upload files n°9/2287 in 0s (0.39%)
 - Upload files n°10/2287 in 0s (0.44%)
 - Upload files n°11/2287 in 0s (0.48%)
 - Upload files n°12/2287 in 0s (0.52%)
 - Upload files n°13/2287 in 0s (0.57%)
 - Upload files n°14/2287 in 0s (0.61%)
 - Upload files n°15/2287 in 0s (0.66%)
 - Upload files n°16/2287 in 0s (0.7%)
 - Upload files n°17/2287 in 0s (0.74%)
 - Upload files n°18/2287 in 0s (0.79%)
 - Upload files n°19/2287 in 0s (0.83%)
 - Upload files n°20/2287 in 0s (0.87%)
 - Upload files n°21/2287 in 0s (0.92%)
 - Upload files n°22/2287 in 0s (0.96%)
 - Upload files n°23/2287 in 0s (1.01%)
 - Upload fi

## Filtrer les attributs

On crée une copie de la base de données originale en supprimant les attributs qui ne nous seront pas utiles pour l'analyse.

Ce filtrage des attributs permettra d'accélérer les requêtes par la suite.

Cette approche permet de conserver les données initialise, sans les modifier. Ainsi, si nous désirerions utiliser un attribut qui n'a pas était importé, on pourra se rebaser sur la première collection et non les fichiers JSON originaux. Nous ne perdrons ainsi aucune donnée.

On va sauvegarder les tweets filtrés dans la collection tweets

In [7]:
tweets_collection = db["tweets"]

On supprime toute la collection pour supprimer par la même occasion les données qu'elle contient.

In [8]:
tweets_collection.drop()

On définit le filtre que l'on va utiliser pour conserver les attributs nécessaires à l'analyse.

In [9]:
tweet_features_to_keep = ["created_date","id","text","entities","favorite_count","lang","possibly_sensitive","retweet_count","source"]
user_features_to_keep = ["friends_count","favourites_count","created_at","id","verified","followers_count","statuses_count"]
def create_filter():
    filterMongodb = {}
    for tweet_feature in tweet_features_to_keep:
        filterMongodb[tweet_feature] = 1
    for user_feature in user_features_to_keep:
        filterMongodb[f"user.{user_feature}"] = 1
    return filterMongodb

In [10]:
filterMongodb = create_filter()

On crée la copie avec les attributs filtrés

In [11]:
BATCH_SIZE = 100_000
page = 0
while True:
    print(f"Page n°{page}")
    docs = dataset_collection.find({}, filterMongodb).skip(page * BATCH_SIZE).limit(BATCH_SIZE)
    docs = [doc for doc in docs]
    if len(docs) == 0:
        print("FIN !")
        break
    tweets_collection.insert_many(docs)
    page += 1


Page n°0
Page n°1
Page n°2
Page n°3
Page n°4
Page n°5
Page n°6
Page n°7
Page n°8
Page n°9
Page n°10
Page n°11
Page n°12
Page n°13
Page n°14
Page n°15
Page n°16
Page n°17
Page n°18
Page n°19
Page n°20
Page n°21
Page n°22
Page n°23
Page n°24
Page n°25
Page n°26
Page n°27
Page n°28
Page n°29
Page n°30
Page n°31
Page n°32
Page n°33
Page n°34
Page n°35
Page n°36
Page n°37
Page n°38
Page n°39
Page n°40
Page n°41
Page n°42
Page n°43
Page n°44
Page n°45
Page n°46
FIN !


## Suppression des doublons

On peut exécuter la cellue suivante afin d'observer si la base de données  comporte ou non des doublons. Ils sont triés par ordre décroissant du nombre total d'apparitions. C'est à dire que les premiers résultats désigne les documents qui apparaissent le plus fréquemment.

In [12]:
res = tweets_collection.aggregate([
    {
        "$group": {
            "_id": "$id",
            "count": { "$sum": 1 }
        }
    },
    {
        "$match": {
            "count": { "$gt": 1 }
        }
    },
    { 
        "$sort" : { 
            "count" : -1 
        } 
    }
])

docs = []
for doc in res:
    print(doc)
    docs.append(doc)

if len(docs) == 0:
    print('Aucun doublon')

{'_id': 1007697967787249665, 'count': 49}
{'_id': 1007369325081251841, 'count': 39}
{'_id': 1007626292810997761, 'count': 39}
{'_id': 1008070034651058176, 'count': 21}
{'_id': 1007408706076794880, 'count': 4}
{'_id': 1007628550634070017, 'count': 4}
{'_id': 1008014342489505792, 'count': 4}
{'_id': 1007109381043843072, 'count': 3}
{'_id': 1007164758246617088, 'count': 3}
{'_id': 1007169278393516032, 'count': 3}
{'_id': 1007173802130731008, 'count': 3}
{'_id': 1007192930606616576, 'count': 3}
{'_id': 1007216834620239872, 'count': 3}
{'_id': 1007219856763043842, 'count': 3}
{'_id': 1007226401576144896, 'count': 3}
{'_id': 1007238227068010496, 'count': 3}
{'_id': 1007259118120210439, 'count': 3}
{'_id': 1007273463847899136, 'count': 3}
{'_id': 1007282018395312129, 'count': 3}
{'_id': 1007282768961679367, 'count': 3}
{'_id': 1007287554079510528, 'count': 3}
{'_id': 1007289062284144640, 'count': 3}
{'_id': 1007290822352498688, 'count': 3}
{'_id': 1007295103646470146, 'count': 3}
{'_id': 1007

On supprime les doublons

In [13]:
pipeline = [
    {"$group": {"_id": "$id", "count": {"$sum": 1}, "dups": {"$addToSet": "$_id"}}},
    {"$match": {"count": {"$gt": 1}}}
]

duplicates = []
for doc in tweets_collection.aggregate(pipeline):
    duplicates.extend(doc["dups"][1:])

tweets_collection.delete_many({"_id": {"$in": duplicates}})

## Création d'indexes 
Afin de rendre les requêtes à suivre plus rapides on crée des indexes au sein de notre base de données mongoDB

In [14]:
tweets_collection.create_index([("id",1)])
tweets_collection.create_index([("user.id",1)])
tweets_collection.create_index([("user.id",1),("id",1)])
tweets_collection.create_index([("user.id",1),("created_date",1)])

'user.id_1_created_date_1'